In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## market 데이터 변수 축소-각 변수별 그래프 그려보고 비슷한 양상을 보이는 변수끼리 묶기

In [2]:
import pandas as pd
import numpy as np

In [3]:
m=pd.read_csv('2019빅데이터페스티벌데이터/market_noDram_final.csv', encoding= 'euc_kr')

In [34]:
m.head()

,날짜,A등급 구리(현물),니켈(현물),알루미늄 99.7%(현물),금괴(일간),미국1년,미국3년,미국10년,국고채권(1년),국고채권(3년),국고채권(10년),브랜트유 가격,원유 WTI,금속
0,2001-01-02,1746.3,6935.0,1552.5,270.6,5.2,4.8,4.9,6.7,6.7,7.2,24.3,27.2,7202.509750
1,2001-01-03,1716.8,6370.0,1515.5,267.5,5.1,4.9,5.2,6.6,6.6,7.2,25.0,28.0,6641.324150
2,2001-01-04,1750.5,6569.0,1519.0,267.7,4.9,4.8,5.0,6.4,6.4,6.9,25.4,28.1,6843.049220
3,2001-01-05,1740.3,6607.0,1527.5,268.2,4.6,4.6,4.9,6.2,6.2,6.7,25.2,28.0,6878.711003
4,2001-01-08,1775.0,6734.0,1578.0,268.3,4.6,4.6,4.9,6.1,6.1,6.6,24.4,27.3,7012.097592


## m에서 'A등급 구리(현물)','니켈(현물)','알루미늄 99.7%(현물)'만 선택해서 새로운 데이터 프레임을 만들고 변수축소 수행

In [16]:
from sklearn.decomposition import PCA

pca=PCA(n_components=1)
m1=m[['A등급 구리(현물)','니켈(현물)','알루미늄 99.7%(현물)']]
m2=pca.fit(m1)
print('PCA component 별 변동성:', pca.explained_variance_ratio_)

PCA component 별 변동성: [0.95585305]


In [6]:
print(pca.components_.T)
pca.components_.T.shape

[[0.19241083]
 [0.98034117]
 [0.04369514]]


(3, 1)

In [7]:
m2.components_.T[0]

array([0.19241083])

## 한 값만 지나치게 영향을 주는 부분이 있는 것 같아 스케일링 시도

In [8]:
m1_1= StandardScaler().fit_transform(m1)

m1_1=pd.DataFrame(m1_1)
m1_1.columns=['A등급 구리(현물)','니켈(현물)','알루미늄 99.7%(현물)']
m1_1.head()

,A등급 구리(현물),니켈(현물),알루미늄 99.7%(현물)
0,-1.594937,-1.108757,-0.901840
1,-1.607458,-1.178171,-0.987416
2,-1.593155,-1.153723,-0.979321
3,-1.597484,-1.149054,-0.959662
4,-1.582756,-1.133451,-0.842862


In [14]:
m2_1=pca.fit(m1_1)
print('PCA component 별 변동성:', pca.explained_variance_ratio_)
print(pca.components_.T)
m2_1.components_.T.shape

PCA component 별 변동성: [0.83067367]
[[0.55586834]
 [0.57356545]
 [0.60169183]]


(3, 1)

## 그러나 스케일링 수행 할 시 세 변수에 대해 설명하는 부분이 부족하다는 판단에 스케일링 없이 차원 축소 하기로 결정함

In [18]:
pca=PCA(n_components=1)
m1=m[['A등급 구리(현물)','니켈(현물)','알루미늄 99.7%(현물)']]
m2=pca.fit(m1)

In [19]:
m2pc=m2.components_.T.reshape(1,3)
m2pc

array([[0.19241083, 0.98034117, 0.04369514]])

## pc의 계수를 통해 변수 축소 진행

In [20]:
금속=[]
for i in range(m1.shape[0]):
    a=m2pc[0,0]*m1.iloc[i,0]
    b=m2pc[0,1]*m1.iloc[i,1]
    c=m2pc[0,2]*m1.iloc[i,2]
    금속.append(a+b+c)


In [21]:
금속=pd.DataFrame(금속)
금속.head()

,0
0,7202.509750
1,6641.324150
2,6843.049220
3,6878.711003
4,7012.097592


In [22]:
m['금속']=금속
m.head()

,날짜,A등급 구리(현물),니켈(현물),알루미늄 99.7%(현물),금괴(일간),미국1년,미국3년,미국10년,국고채권(1년),국고채권(3년),국고채권(10년),브랜트유 가격,원유 WTI,금속
0,2001-01-02,1746.3,6935.0,1552.5,270.6,5.2,4.8,4.9,6.7,6.7,7.2,24.3,27.2,7202.509750
1,2001-01-03,1716.8,6370.0,1515.5,267.5,5.1,4.9,5.2,6.6,6.6,7.2,25.0,28.0,6641.324150
2,2001-01-04,1750.5,6569.0,1519.0,267.7,4.9,4.8,5.0,6.4,6.4,6.9,25.4,28.1,6843.049220
3,2001-01-05,1740.3,6607.0,1527.5,268.2,4.6,4.6,4.9,6.2,6.2,6.7,25.2,28.0,6878.711003
4,2001-01-08,1775.0,6734.0,1578.0,268.3,4.6,4.6,4.9,6.1,6.1,6.6,24.4,27.3,7012.097592


## 그리고 원본 변수들 삭제

In [23]:
market=m.drop(m1,axis=1)

## '브랜트유 가격','원유 WTI' 변수들은 그래프가 아주 흡사한 모습을 보였기에 평균으로 축소

In [24]:
m_4=m[['브랜트유 가격','원유 WTI']]
oil=[]
for i in range(m_4.shape[0]):
    a=(m_4.iloc[i,0]+m_4.iloc[i,1])/2
    oil.append(a)
oil=pd.DataFrame(oil)
market['oil']=oil
market=market.drop(m_4, axis=1)
market.head()

,날짜,금괴(일간),미국1년,미국3년,미국10년,국고채권(1년),국고채권(3년),국고채권(10년),금속,oil
0,2001-01-02,270.6,5.2,4.8,4.9,6.7,6.7,7.2,7202.509750,25.75
1,2001-01-03,267.5,5.1,4.9,5.2,6.6,6.6,7.2,6641.324150,26.50
2,2001-01-04,267.7,4.9,4.8,5.0,6.4,6.4,6.9,6843.049220,26.75
3,2001-01-05,268.2,4.6,4.6,4.9,6.2,6.2,6.7,6878.711003,26.60
4,2001-01-08,268.3,4.6,4.6,4.9,6.1,6.1,6.6,7012.097592,25.85


## 나머지 변수들에도 PCA 수행

In [26]:
m_2=m[['미국1년','미국3년','미국10년']]
m2_=pca.fit(m_2)
print('PCA component 별 변동성:', pca.explained_variance_ratio_)
print(pca.components_.T)
pca.components_.T.shape
m_2pc=m2_.components_.T.reshape(1,3)

PCA component 별 변동성: [0.9335029]
[[0.66895425]
 [0.61128065]
 [0.42289027]]


In [28]:
미국n년=[]
for i in range(m_2.shape[0]):
    a=m_2pc[0,0]*m_2.iloc[i,0]
    b=m_2pc[0,1]*m_2.iloc[i,1]
    c=m_2pc[0,2]*m_2.iloc[i,2]
    미국n년.append(a+b+c)
미국n년=pd.DataFrame(미국n년)
market=market.drop(m_2,axis=1)
market['미국n년']=미국n년

In [29]:
market.head()

,날짜,금괴(일간),국고채권(1년),국고채권(3년),국고채권(10년),금속,oil,미국n년
0,2001-01-02,270.6,6.7,6.7,7.2,7202.509750,25.75,8.484872
1,2001-01-03,267.5,6.6,6.6,7.2,6641.324150,26.50,8.605971
2,2001-01-04,267.7,6.4,6.4,6.9,6843.049220,26.75,8.326474
3,2001-01-05,268.2,6.2,6.2,6.7,6878.711003,26.60,7.961243
4,2001-01-08,268.3,6.1,6.1,6.6,7012.097592,25.85,7.961243


In [30]:
m_3=m[['국고채권(1년)','국고채권(3년)','국고채권(10년)']]
m3_=pca.fit(m_3)
print('PCA component 별 변동성:', pca.explained_variance_ratio_)
print(pca.components_.T)
pca.components_.T.shape
m_3pc=m3_.components_.T.reshape(1,3)

PCA component 별 변동성: [0.9702223]
[[0.54440254]
 [0.57661956]
 [0.60920912]]


In [31]:
국고=[]
for i in range(m_3.shape[0]):
    a=m_3pc[0,0]*m_3.iloc[i,0]
    b=m_3pc[0,1]*m_3.iloc[i,1]
    c=m_3pc[0,2]*m_3.iloc[i,2]
    국고.append(a+b+c)
국고=pd.DataFrame(국고)
market=market.drop(m_3,axis=1)
market['국고채권(n년)']=국고

## 모두 수행 후 소수점 첫째자리까지만 남도록

In [32]:
market=market.round(1)

In [33]:
market.head()

,날짜,금괴(일간),금속,oil,미국n년,국고채권(n년)
0,2001-01-02,270.6,7202.5,25.8,8.5,11.9
1,2001-01-03,267.5,6641.3,26.5,8.6,11.8
2,2001-01-04,267.7,6843.0,26.8,8.3,11.4
3,2001-01-05,268.2,6878.7,26.6,8.0,11.0
4,2001-01-08,268.3,7012.1,25.8,8.0,10.9


In [96]:
market.to_csv('market_pca_final.csv', encoding='euc-kr')